# 多智能体分散式发言人选择

本笔记展示了如何实现一个多智能体模拟，其中没有固定的发言时间表，而是由智能体自行决定谁发言。我们可以通过让每个智能体竞标发言来实现这一点。竞标最高的智能体将获得发言权。

下面的示例展示了一个虚构的总统辩论，演示了如何实现这一点。

## 导入与 LangChain 相关的模块

In [1]:
from typing import Callable, List  # 导入需要的模块

import tenacity  # 导入需要的模块
from langchain.output_parsers import RegexParser  # 从langchain.output_parsers模块中导入RegexParser类
from langchain.prompts import PromptTemplate  # 从langchain.prompts模块中导入PromptTemplate类
from langchain.schema import (  # 从langchain.schema模块中导入HumanMessage和SystemMessage类
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI  # 从langchain_openai模块中导入ChatOpenAI类

## `DialogueAgent` 和 `DialogueSimulator` 类
我们将使用在[多人龙与地下城](https://python.langchain.com/en/latest/use_cases/agent_simulations/multi_player_dnd.html)中定义的相同的 `DialogueAgent` 和 `DialogueSimulator` 类。

In [2]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        将聊天模型应用于消息历史并返回消息字符串
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        将{name}说的{message}连接到消息历史中
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        用{name}的{message}开始对话
        """
        for agent in self.agents:
            agent.receive(name, message)

        # 增加时间步数
        self._step += 1

    def step() -> tuple[str, str]:
        # 1. 选择下一个发言者
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. 下一个发言者发送消息
        message = speaker.send()

        # 3. 每个人接收消息
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. 增加时间步数
        self._step += 1

        return speaker.name, message

## `BiddingDialogueAgent` 类
我们定义了一个 `DialogueAgent` 的子类，该子类具有一个 `bid()` 方法，根据消息历史和最近的消息生成一个出价。

In [3]:
# 定义一个名为BiddingDialogueAgent的类，继承自DialogueAgent类
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        # 调用父类的初始化方法
        super().__init__(name, system_message, model)
        # 初始化竞价模板
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        请求聊天模型输出一个竞价发言
        """
        # 根据竞价模板和历史消息生成输入提示
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        # 调用模型生成竞价字符串
        bid_string = self.model.invoke([SystemMessage(content=prompt)]).content
        return bid_string

## 定义参与者和辩论主题

In [4]:
# 定义一个包含角色名称的列表
character_names = ["Donald Trump", "Kanye West", "Elizabeth Warren"]

# 定义一个话题
topic = "transcontinental high speed rail"

# 定义一个词数限制
word_limit = 50

## 生成系统消息

In [5]:
game_description = f"""Here is the topic for the presidential debate: {topic}.
The presidential candidates are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each presidential candidate."
)



def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            请用不超过{word_limit}个字的创造性描述回答总统候选人{character_name}的个性特点。
            直接对{character_name}说。
            不要添加其他内容。"""
        ),
    ]
    character_description = ChatOpenAI(temperature=1.0)(
        character_specifier_prompt
    ).content
    return character_description


def generate_character_header(character_name, character_description):
    return f"""{game_description}
您的名字是{character_name}。
您是一位总统候选人。
您的描述如下：{character_description}
您正在辩论的主题是：{topic}。
您的目标是尽可能地创造性，让选民认为您是最好的候选人。
"""


def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
您将以{character_name}的风格发言，并夸大他们的个性。
您将提出与{topic}相关的创意。
不要一遍又一遍地说同样的话。
以{character_name}的第一人称视角发言。
在描述自己的身体动作时，请用“*”将描述包围起来。
不要改变角色！
不要从其他人的角度发言。
只从{character_name}的角度发言。
当您从自己的角度发言结束时，请停止发言。
永远不要忘记将您的回答保持在{word_limit}个字以内！
不要添加其他内容。
    """
        )
    )


character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_headers)
    for character_name, character_headers in zip(character_names, character_headers)
]

In [6]:
# 使用zip函数同时遍历四个列表
for (
    character_name,  # 角色名称
    character_description,  # 角色描述
    character_header,  # 角色头部
    character_system_message,  # 角色系统消息
) in zip(
    character_names,  # 角色名称列表
    character_descriptions,  # 角色描述列表
    character_headers,  # 角色头部列表
    character_system_messages,  # 角色系统消息列表
):
    print(f"\n\n{character_name} Description:")  # 打印角色名称和描述标题
    print(f"\n{character_description}")  # 打印角色描述
    print(f"\n{character_header}")  # 打印角色头部
    print(f"\n{character_system_message.content}")  # 打印角色系统消息内容



Donald Trump Description:

Donald Trump, you are a bold and outspoken individual, unafraid to speak your mind and take on any challenge. Your confidence and determination set you apart and you have a knack for rallying your supporters behind you.

Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Donald Trump, Kanye West, Elizabeth Warren.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump, you are a bold and outspoken individual, unafraid to speak your mind and take on any challenge. Your confidence and determination set you apart and you have a knack for rallying your supporters behind you.
You are debating the topic: transcontinental high speed rail.
Your goal is to be as creative as possible and make the voters think you are the best candidate.


Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Dona

## 用于Bids的输出解析器
我们要求代理输出一个竞标发言。但是由于代理是输出字符串的LLMs，我们需要：
1. 定义一个格式，让他们能够产生输出
2. 解析他们的输出

我们可以继承[RegexParser](https://github.com/langchain-ai/langchain/blob/master/langchain/output_parsers/regex.py)来实现我们自己的竞标输出解析器。

In [7]:
# 定义一个名为BidOutputParser的类，继承自RegexParser类
class BidOutputParser(RegexParser):
    # 定义一个名为get_format_instructions的方法，返回一个字符串
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."

# 创建一个BidOutputParser的实例，传入正则表达式、输出键列表和默认输出键
bid_parser = BidOutputParser(
    regex=r"<(\d+)>", output_keys=["bid"], default_output_key="bid"
)

## 生成竞标系统消息
这是受到[生成式代理](https://arxiv.org/pdf/2304.03442.pdf)中使用LLM确定记忆重要性的提示的启发。这将使用我们的`BidOutputParser`的格式化指令。

In [8]:
# 生成角色竞标模板函数
def generate_character_bidding_template(character_header):
    # 使用f-string格式化字符串，将character_header插入模板中
    bidding_template = f"""{character_header}

{{message_history}}

在1到10的范围内，1表示不矛盾，10表示非常矛盾，请评价以下信息与您的观点之间的矛盾程度。

{{recent_message}}

{bid_parser.get_format_instructions()}
不做其他操作。
    """
    return bidding_template


# 生成角色竞标模板列表
character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

In [9]:
# 循环遍历两个列表，分别是character_names和character_bidding_templates
for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    # 打印角色名称
    print(f"{character_name} Bidding Template:")
    # 打印竞标模板
    print(bidding_template)

Donald Trump Bidding Template:
Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Donald Trump, Kanye West, Elizabeth Warren.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump, you are a bold and outspoken individual, unafraid to speak your mind and take on any challenge. Your confidence and determination set you apart and you have a knack for rallying your supporters behind you.
You are debating the topic: transcontinental high speed rail.
Your goal is to be as creative as possible and make the voters think you are the best candidate.


```
{message_history}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{recent_message}
```

Your response should be an integer delimited by angled brackets, like this: <int>.
Do nothing else.
    
Kanye West Bidding Templat

## 使用LLM来深入探讨辩论主题

In [10]:
# 代码注释

# 定义一个包含提示信息的列表，用于指定话题
topic_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),  # 系统消息：您可以使任务更具体。
    HumanMessage(
        content=f"""{game_description}
        
        You are the debate moderator.
        Please make the debate topic more specific. 
        Frame the debate topic as a problem to be solved.
        Be creative and imaginative.
        Please reply with the specified topic in {word_limit} words or less. 
        Speak directly to the presidential candidates: {*character_names,}.
        Do not add anything else."""
    ),  # 人类消息：您是辩论主持人。请将辩论话题更具体化。将辩论话题构建为一个需要解决的问题。要有创造力和想象力。请用不超过{word_limit}个字回复指定的话题。直接对总统候选人说话：{*character_names,}。不要添加其他内容。
]

# 使用ChatOpenAI模型生成指定的话题
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

# 打印原始话题
print(f"Original topic:\n{topic}\n")

# 打印详细话题
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
transcontinental high speed rail

Detailed topic:
The topic for the presidential debate is: "Overcoming the Logistics of Building a Transcontinental High-Speed Rail that is Sustainable, Inclusive, and Profitable." Donald Trump, Kanye West, Elizabeth Warren, how will you address the challenges of building such a massive transportation infrastructure, dealing with stakeholders, and ensuring economic stability while preserving the environment?



## 定义说话者选择函数
最后，我们将定义一个说话者选择函数`select_next_speaker`，该函数接受每个代理人的出价，并选择出价最高的代理人（如果出价相同，则随机选择）。

我们将定义一个`ask_for_bid`函数，该函数使用我们之前定义的`bid_parser`来解析代理人的出价。我们将使用`tenacity`来装饰`ask_for_bid`函数，以便在代理人的出价无法正确解析时进行多次重试，并在最大尝试次数后产生默认出价为0。

In [11]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),  # 设置最大重试次数为2次
    wait=tenacity.wait_none(),  # 重试之间没有等待时间
    retry=tenacity.retry_if_exception_type(ValueError),  # 仅在出现 ValueError 异常时进行重试
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),  # 在每次重试之前打印出现的 ValueError 异常信息
    retry_error_callback=lambda retry_state: 0,  # 当所有重试次数用尽时，默认返回值为0
)  # 当所有重试次数用尽时，默认返回值为0
def ask_for_bid(agent) -> str:
    """
    请求代理的出价并将出价解析为正确的格式。
    """
    bid_string = agent.bid()  # 调用代理的出价方法获取出价字符串
    bid = int(bid_parser.parse(bid_string)["bid"])  # 解析出价字符串并将其转换为整数
    return bid  # 返回出价

In [12]:
import numpy as np

def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)  # 向每个代理商询问出价
        bids.append(bid)

    # 在多个出价相同时随机选择一个代理商
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("出价：")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} 的出价: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"选择的代理商: {selected_name}")
    print("\n")
    return idx

## 主循环

In [13]:
# 创建一个空列表用于存储角色
characters = []

# 使用zip函数将character_names、character_system_messages和character_bidding_templates三个列表进行循环迭代
# character_name表示角色名称，character_system_message表示角色系统消息，bidding_template表示竞标模板
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    # 创建一个BiddingDialogueAgent对象，并将其添加到characters列表中
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
            bidding_template=bidding_template,
        )
    )

In [15]:
# 设置最大迭代次数
max_iters = 10
# 初始化迭代计数器
n = 0

# 创建对话模拟器实例，传入角色列表和选择下一个发言者的函数
simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
# 重置对话模拟器状态
simulator.reset()
# 注入指定话题给"Debate Moderator"角色
simulator.inject("Debate Moderator", specified_topic)
# 打印指定话题
print(f"(Debate Moderator): {specified_topic}")
print("\n")

# 迭代对话模拟器直到达到最大迭代次数
while n < max_iters:
    # 模拟器进行一步对话，返回发言者和发言内容
    name, message = simulator.step()
    # 打印发言者和发言内容
    print(f"({name}): {message}")
    print("\n")
    # 更新迭代计数器
    n += 1

(Debate Moderator): The topic for the presidential debate is: "Overcoming the Logistics of Building a Transcontinental High-Speed Rail that is Sustainable, Inclusive, and Profitable." Donald Trump, Kanye West, Elizabeth Warren, how will you address the challenges of building such a massive transportation infrastructure, dealing with stakeholders, and ensuring economic stability while preserving the environment?


Bids:
	Donald Trump bid: 7
	Kanye West bid: 5
	Elizabeth Warren bid: 1
Selected: Donald Trump


(Donald Trump): Let me tell you, folks, I know how to build big and I know how to build fast. We need to get this high-speed rail project moving quickly and efficiently. I'll make sure we cut through the red tape and get the job done. And let me tell you, we'll make it profitable too. We'll bring in private investors and make sure it's a win-win for everyone. *gestures confidently*


Bids:
	Donald Trump bid: 2
	Kanye West bid: 8
	Elizabeth Warren bid: 10
Selected: Elizabeth Warren

